In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("msambare/fer2013")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'fer2013' dataset.
Path to dataset files: /kaggle/input/fer2013


In [ ]:
cd /kaggle/input/fer2013

/kaggle/input/fer2013


In [ ]:
ls

test/  train/


In [ ]:
import os

def walk_through(dir_path):
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [ ]:
walk_through(path)


There are 2 directories and 0 images in '/kaggle/input/fer2013'.
There are 7 directories and 0 images in '/kaggle/input/fer2013/test'.
There are 0 directories and 831 images in '/kaggle/input/fer2013/test/surprise'.
There are 0 directories and 1024 images in '/kaggle/input/fer2013/test/fear'.
There are 0 directories and 958 images in '/kaggle/input/fer2013/test/angry'.
There are 0 directories and 1233 images in '/kaggle/input/fer2013/test/neutral'.
There are 0 directories and 1247 images in '/kaggle/input/fer2013/test/sad'.
There are 0 directories and 111 images in '/kaggle/input/fer2013/test/disgust'.
There are 0 directories and 1774 images in '/kaggle/input/fer2013/test/happy'.
There are 7 directories and 0 images in '/kaggle/input/fer2013/train'.
There are 0 directories and 3171 images in '/kaggle/input/fer2013/train/surprise'.
There are 0 directories and 4097 images in '/kaggle/input/fer2013/train/fear'.
There are 0 directories and 3995 images in '/kaggle/input/fer2013/train/angry'

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
train_dir = '/kaggle/input/fer2013/train'
test_dir = '/kaggle/input/fer2013/test'

In [ ]:
IMG_HEIGHT=48
IMG_WIDTH=48
BATCH_SIZE=32


In [ ]:
train_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(IMG_HEIGHT,IMG_WIDTH),
    color_mode='grayscale',

    batch_size=BATCH_SIZE
)

Found 28709 files belonging to 7 classes.
Using 22968 files for training.


In [ ]:
validation_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(IMG_HEIGHT,IMG_WIDTH),
    color_mode='grayscale',

    batch_size=BATCH_SIZE
)

Found 28709 files belonging to 7 classes.
Using 5741 files for validation.


In [ ]:
class_names=train_dataset.class_names
class_names

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [ ]:
num_classes=len(class_names)
num_classes


7

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
from tensorflow.keras import layers, models

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.1),
])

In [ ]:
model = models.Sequential([
    layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 1)),

    data_augmentation,
    layers.Rescaling(1./255),

    # Block 1
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.3),

    # Block 2
    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.4),

    # Block 3
    layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.5),

    # Classifier Head
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.6),
    layers.Dense(num_classes, activation='softmax')
])


In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 48, 48, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 48, 48, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 48, 48, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,179,77

 Total params: 2,329,031 (8.88 MB)

 Trainable params: 2,326,983 (8.88 MB)

 Non-trainable params: 2,048 (8.00 KB)

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

In [ ]:
history=model.fit(
    train_dataset,
    epochs=25,
    validation_data=validation_dataset,
    callbacks=[early_stopping]
)

Epoch 1/25
718/718 ━━━━━━━━━━━━━━━━━━━━ 46s 50ms/step - accuracy: 0.1976 - loss: 2.4675 - val_accuracy: 0.3416 - val_loss: 1.6487
Epoch 2/25
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.3087 - loss: 1.7399 - val_accuracy: 0.3635 - val_loss: 1.6477
Epoch 3/25
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.3886 - loss: 1.5737 - val_accuracy: 0.3762 - val_loss: 1.6367
Epoch 4/25
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.4403 - loss: 1.4580 - val_accuracy: 0.4909 - val_loss: 1.3334
Epoch 5/25
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.4624 - loss: 1.4003 - val_accuracy: 0.4952 - val_loss: 1.3034
Epoch 6/25
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.4875 - loss: 1.3535 - val_accuracy: 0.4910 - val_loss: 1.3162
Epoch 7/25
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.4995 - loss: 1.3104 - val_accuracy: 0.5429 - val_loss: 1.2165
Epoch 8/25
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.5151 - loss: 1.2828 - 

In [ ]:
history=model.fit(
    train_dataset,
    epochs=15,
    validation_data=validation_dataset,
    callbacks=[early_stopping]
)

Epoch 1/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.5921 - loss: 1.0824 - val_accuracy: 0.6044 - val_loss: 1.0592
Epoch 2/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.5916 - loss: 1.0850 - val_accuracy: 0.6213 - val_loss: 1.0246
Epoch 3/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.5992 - loss: 1.0658 - val_accuracy: 0.6178 - val_loss: 1.0172
Epoch 4/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.6039 - loss: 1.0553 - val_accuracy: 0.6281 - val_loss: 1.0105
Epoch 5/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.6066 - loss: 1.0512 - val_accuracy: 0.6306 - val_loss: 1.0088
Epoch 6/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.6101 - loss: 1.0441 - val_accuracy: 0.6152 - val_loss: 1.0294
Epoch 7/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.6179 - loss: 1.0288 - val_accuracy: 0.6213 - val_loss: 1.0331
Epoch 8/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.6078 - loss: 1.0371 - 

In [ ]:
history=model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=[early_stopping]
)

Epoch 1/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.6271 - loss: 1.0001 - val_accuracy: 0.6293 - val_loss: 1.0051
Epoch 2/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.6304 - loss: 0.9952 - val_accuracy: 0.6361 - val_loss: 1.0013
Epoch 3/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.6273 - loss: 0.9877 - val_accuracy: 0.6130 - val_loss: 1.0355
Epoch 4/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 28s 39ms/step - accuracy: 0.6294 - loss: 0.9895 - val_accuracy: 0.6386 - val_loss: 0.9879
Epoch 5/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.6348 - loss: 0.9844 - val_accuracy: 0.6321 - val_loss: 1.0046
Epoch 6/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.6383 - loss: 0.9773 - val_accuracy: 0.6342 - val_loss: 1.0054
Epoch 7/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.6409 - loss: 0.9674 - val_accuracy: 0.6407 - val_loss: 0.9812
Epoch 8/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.6340 - loss: 0.9766 - 

In [ ]:
history=model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=[early_stopping]
)


Epoch 1/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.6346 - loss: 0.9766 - val_accuracy: 0.6347 - val_loss: 0.9894
Epoch 2/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 28s 39ms/step - accuracy: 0.6473 - loss: 0.9623 - val_accuracy: 0.6405 - val_loss: 0.9919
Epoch 3/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.6369 - loss: 0.9623 - val_accuracy: 0.6300 - val_loss: 1.0244
Epoch 4/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.6456 - loss: 0.9620 - val_accuracy: 0.6382 - val_loss: 0.9936
Epoch 5/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.6449 - loss: 0.9562 - val_accuracy: 0.6316 - val_loss: 1.0121


In [ ]:
history=model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=[early_stopping]
)


Epoch 1/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.6373 - loss: 0.9767 - val_accuracy: 0.6391 - val_loss: 0.9847
Epoch 2/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.6417 - loss: 0.9618 - val_accuracy: 0.6443 - val_loss: 0.9879
Epoch 3/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.6434 - loss: 0.9594 - val_accuracy: 0.6318 - val_loss: 1.0019
Epoch 4/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.6431 - loss: 0.9506 - val_accuracy: 0.6422 - val_loss: 0.9830
Epoch 5/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.6443 - loss: 0.9504 - val_accuracy: 0.6325 - val_loss: 1.0009
Epoch 6/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.6445 - loss: 0.9537 - val_accuracy: 0.6328 - val_loss: 0.9985
Epoch 7/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.6464 - loss: 0.9419 - val_accuracy: 0.6353 - val_loss: 0.9937
Epoch 8/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.6533 - loss: 0.9353 - 

In [ ]:
google.colab.files.download('/tmp/emotion_detection_model.keras')

NameError: name 'google' is not defined

In [ ]:
from google.colab import files

In [ ]:
files.download('/tmp/emotion_detection_model.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>